In [ ]:
import kagglehub

path = kagglehub.dataset_download("yusufberksardoan/traffic-detection-project")

print("Path to dataset files:", path)

In [ ]:
import os

print("Dataset files:", os.listdir(path))


In [ ]:
import yaml

yaml_path = os.path.join(path, "data.yaml")

with open(yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)

print("Dataset Info:", data_config)


In [ ]:
image_folder = os.path.join(path, "train/images")
print("Available images:", os.listdir(image_folder))

In [ ]:
image_files = os.listdir(image_folder)  
image_path = os.path.join(image_folder, image_files[0]) 

print("Using Image:", image_path)

In [ ]:
import cv2
import matplotlib.pyplot as plt

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.axis("off")
plt.title("Sample Traffic Image")
plt.show()

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt

In [ ]:
train: '/content/yolov5/1/train/images'
val: '/content/yolov5/1/valid/images'
test: '/content/yolov5/1/test/images'

nc: 5
names: ['bicycle', 'bus', 'car', 'motorbike', 'person']

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov5s.pt')

results = model.train(data=yaml_path, epochs=30, imgsz=640,batch=64)

In [ ]:
import os

weights_path = "/content/runs/detect/train/weights"
if os.path.exists(weights_path):
    print("Contents of 'weights':", os.listdir(weights_path))
else:
    print("No 'weights' folder found. Check if training completed!")


In [ ]:
from ultralytics import YOLO

model = YOLO("/content/runs/detect/train/weights/best.pt")
print("Model loaded successfully!")


In [ ]:
import os

video_path = "/content/4740223-hd_1920_1080_25fps.mp4"
if os.path.exists(video_path):
    print("Video found!")
else:
    print("Video not found. Check the path.")


In [ ]:
import os
import shutil

output_dir = "frames_1"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)  
    os.makedirs(output_dir) 
    print(f"Cleared all files from {output_dir}.")
else:
    print(f"Folder {output_dir} does not exist.")


In [ ]:
import cv2
import os

video_path = "/content/4740223-hd_1920_1080_25fps.mp4" 
output_dir = "frames_1"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) 
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_rate == 0: 
        frame_filename = os.path.join(output_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_filename, frame)

    frame_count += 1

cap.release()
print(f"Frames extracted successfully! Saved in: {output_dir}")


In [ ]:
import os

output_dir = "frames_1"
frames = os.listdir(output_dir)

if frames:
    print(f" {len(frames)} frames found in {output_dir}:")
    print(frames[:10]) 
else:
    print("No frames found. Check if the extraction worked correctly.")


In [ ]:
import cv2
import os
from ultralytics import YOLO

model = YOLO("yolo11n.pt") 

frames_dir = "frames_1"
output_dir = "detected_frames_1"

os.makedirs(output_dir, exist_ok=True)

for frame_name in sorted(os.listdir(frames_dir)):
    frame_path = os.path.join(frames_dir, frame_name)
    frame = cv2.imread(frame_path)

    if frame is None:
        continue

    results = model(frame)

    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    output_path = os.path.join(output_dir, frame_name)
    cv2.imwrite(output_path, frame)

print("Processing complete. Detected frames saved in:", output_dir)


In [ ]:
frame_files = sorted(os.listdir(output_dir))
frame_path = os.path.join(output_dir, frame_files[0])

frame = cv2.imread(frame_path)
height, width, _ = frame.shape

output_video_path = "/content/output_1.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = 30  

out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

for frame_name in frame_files:
    frame = cv2.imread(os.path.join(output_dir, frame_name))
    out.write(frame)

out.release()
print("Final video saved as:", output_video_path)

In [ ]:
import os

def find_file(root_dir, filename):
    for dirpath, _, filenames in os.walk(root_dir):
        if filename in filenames:
            return os.path.join(dirpath, filename)
    return None

video_file = find_file("/content", "output_1.mp4")

if video_file:
    print("Video found at:", video_file)
else:
    print("output_1.mp4 not found!")


In [ ]:
results = model(frame, conf=0.2, iou=0.45)

In [ ]:
for result in results:
    print(result.names)

In [ ]:
import wandb

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from ultralytics import NAS, YOLO


model = YOLO('yolov8l.pt') 

In [ ]:
model.train(
    data='/root/.cache/kagglehub/datasets/yusufberksardoan/traffic-detection-project/versions/1/data.yaml',  # Path to the .yaml file describing the dataset
    epochs=10, 
    batch=16, 
    imgsz=640, 
    workers=2, 
    device=0, 
    patience=50,  
    save_period=10,  
    save=True,  
    project='traffic_detection',
    name='yolov8_best',
    verbose=True 
)

In [ ]:
import torch
import os
import cv2
import pandas as pd
from tqdm import tqdm
import random, os

In [ ]:
tpaths=[]
images_dir = os.path.join(path, 'test', 'images')
for dirname, _, filenames in os.walk(images_dir):
    for filename in filenames:
        tpaths+=[(os.path.join(dirname, filename))]
tpaths2=random.sample(tpaths,10)

In [ ]:
results = model.predict(source=tpaths2, conf=0.5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

results = model(tpaths2)


result = results[8]

image = cv2.imread(tpaths2[8])

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


boxes = result.boxes  
names = result.names   

for box in boxes.data:  
    x1, y1, x2, y2, conf, cls = box 
    class_name = names[int(cls)]
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    label = f"{class_name} {conf:.2f}"
    cv2.putText(image, label, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 10))
plt.imshow(image_rgb)
plt.axis('off') 
plt.show()

In [ ]:
import torch
import os
import cv2
import pandas as pd
from tqdm import tqdm

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

vehicle_classes = ['bicycle', 'car', 'motorcycle', 'bus', 'truck']

dataset_path = '/root/.cache/kagglehub/datasets/yusufberksardoan/traffic-detection-project/versions/1'  # Corrected path
images_path = os.path.join(dataset_path, 'train', 'images')  # Change 'test' to 'train' or 'valid' accordingly

output_data = []

for image_name in tqdm(os.listdir(images_path)):
    image_path = os.path.join(images_path, image_name)

    img = cv2.imread(image_path)

    results = model(img)

    detections = results.pandas().xyxy[0]  

    for index, row in detections.iterrows():
        class_name = row['name']
        if class_name in vehicle_classes:
            vehicle_counts[class_name] += 1

    output_data.append([image_name] + list(vehicle_counts.values()))

columns = ['image_name'] + vehicle_classes
df = pd.DataFrame(output_data, columns=columns)

csv_output_path = 'vehicle_counts.csv'
df.to_csv(csv_output_path, index=False)

print(f'CSV file with vehicle counts saved to {csv_output_path}')

In [ ]:
print(model)

num_layers = sum(1 for _ in model.modules())
print(f'Total number of layers: {num_layers}')

In [ ]:
cctv_cameras = ['aguanambi', 'image', 'screenshot', 'duque', 'ant_sales']

def get_cctv_group(image_name):
    for cctv in cctv_cameras:
        if image_name.startswith(cctv):
            return cctv
    return 'unknown' 
csv['cctv_group'] = csv['image_name'].apply(get_cctv_group)

grouped_df = csv.groupby('cctv_group').sum().reset_index()

In [ ]:
csv

In [ ]:
csv[csv['cctv_group'] == 'aguanambi'].shape

In [ ]:
csv[csv['cctv_group'] == 'ant_sales'].shape

In [ ]:
csv[csv['cctv_group'] == 'duque'].shape

In [ ]:
csv[csv['cctv_group'] == 'image'].shape

In [ ]:
csv[csv['cctv_group'] == 'screenshot'].shape

In [ ]:
grouped_df

In [ ]:
grouped_df['cctv_group'].unique()

In [ ]:
import numpy as np

timestamps = pd.date_range(start='2026-01-07', periods=40, freq='T')

grouped_dfs = {}
for group_name in csv['cctv_group'].unique():
    group_df = csv[csv['cctv_group'] == group_name].sample(n=40, random_state=42, replace=False).reset_index(drop=True)
    group_df['timestamp'] = timestamps 
    grouped_dfs[group_name] = group_df

for group_name, group_df in grouped_dfs.items():
    output_path = f'{group_name}_dataset.csv' 
    group_df.to_csv(output_path, index=False)
    print(f'Saved {group_name} dataset to {output_path}')

In [ ]:
aguanambi_df = pd.read_csv('/content/aguanambi_dataset.csv')
ant_sales_df = pd.read_csv('/content/ant_sales_dataset.csv')
duque_df = pd.read_csv('/content/duque_dataset.csv')
screenshot_df = pd.read_csv('/content/screenshot_dataset.csv')
image_df = pd.read_csv('/content/image_dataset.csv')

In [ ]:
aguanambi_df.head()

In [ ]:
ant_sales_df.head()

In [ ]:
duque_df.head()

In [ ]:
screenshot_df.head()

In [ ]:
image_df.head()

In [ ]:
df = pd.concat([aguanambi_df, ant_sales_df, duque_df, screenshot_df, image_df], ignore_index=True)
df.to_csv('processed_traffic_data.csv')

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df['total_vehicles'] = df[['bicycle', 'car', 'motorcycle', 'bus', 'truck']].sum(axis=1)

df_grouped = df.groupby(['cctv_group', 'timestamp']).agg(total_vehicles=('total_vehicles', 'sum')).reset_index()

congestion_threshold = 10
df_grouped['congestion'] = df_grouped['total_vehicles'] > congestion_threshold

peak_hours = df_grouped[df_grouped['total_vehicles'] > congestion_threshold].groupby('timestamp').agg(
    total_traffic=('total_vehicles', 'sum')).reset_index()

line_fig = px.line(df_grouped, x='timestamp', y='total_vehicles', color='cctv_group',
                   title='Traffic Flow Over Time at Different Junctions')

heatmap_fig = px.density_heatmap(df_grouped, x='timestamp', y='cctv_group', z='total_vehicles',
                                 title='Traffic Intensity Heatmap')

bar_fig = px.bar(peak_hours, x='timestamp', y='total_traffic',
                 title='Peak Congestion Hours Across All Junctions')



In [ ]:
line_fig.show()

In [ ]:
 heatmap_fig.show(), bar_fig.show()

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import IsolationForest
import plotly.express as px


df['total_vehicles'] = df[['bicycle', 'car', 'motorcycle', 'bus', 'truck']].sum(axis=1)

df_grouped = df.groupby('timestamp').agg(total_vehicles=('total_vehicles', 'sum')).reset_index()

model = ARIMA(df_grouped['total_vehicles'], order=(5, 1, 0))  
model_fit = model.fit()

forecast = model_fit.forecast(steps=10)

forecast_df = pd.DataFrame({
    'timestamp': pd.date_range(start=df_grouped['timestamp'].max(), periods=10, freq='min'),
    'total_vehicles': forecast
})

df_grouped['z_score'] = (df_grouped['total_vehicles'] - df_grouped['total_vehicles'].mean()) / df_grouped['total_vehicles'].std()
df_grouped['anomaly'] = np.where(np.abs(df_grouped['z_score']) > 2, True, False) 


df_recommendation = forecast_df[forecast_df['total_vehicles'] == forecast_df['total_vehicles'].min()]

forecast_fig = px.line(pd.concat([df_grouped, forecast_df]), x='timestamp', y='total_vehicles',
                       title='Predicted Traffic Over Time (ARIMA)', markers=True)

anomaly_fig = px.scatter(df_grouped, x='timestamp', y='total_vehicles', color='anomaly',
                         title='Anomaly Detection in Traffic Flow')

print("Recommended time to travel based on predicted traffic:")
print(df_recommendation)

(forecast_fig.show(), anomaly_fig.show())

In [ ]:
from prophet import Prophet
from plotly.subplots import make_subplots

In [ ]:
from dash import Dash, dcc, html, Input, Output

In [ ]:
df['cctv_group'].unique()

In [ ]:
import plotly.graph_objects as go


df['total_vehicles'] = df[['bicycle', 'car', 'motorcycle', 'bus', 'truck']].sum(axis=1)

def prepare_data_for_prophet(df, junction):
    df_junction = df[df['cctv_group'] == junction][['timestamp', 'total_vehicles']].copy()
    df_junction.columns = ['ds', 'y'] 
    return df_junction

models = {}
junctions = df['cctv_group'].unique()

for junction in junctions:
    df_junction = prepare_data_for_prophet(df, junction)
    model = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
    model.add_seasonality(name='min', period=2, fourier_order=10)
    model.fit(df_junction)
    models[junction] = model

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Traffic Forecast and Vehicle Count at Different Junctions"),

    dcc.Dropdown(
        id='junction-dropdown',
        options=[{'label': junction, 'value': junction} for junction in junctions],
        value=junctions[0],
        style={'width': '50%'}
    ),

    dcc.Slider(
        id='points-slider',
        min=5,
        max=50,
        step=5,
        value=30,  
        marks={i: f'{i} points' for i in range(5, 50, 10)},
        tooltip={'always_visible': True}
    ),

    html.Label("Select Timestamp:"),
    dcc.DatePickerSingle(
        id='timestamp-picker',
        min_date_allowed=df['timestamp'].min(),
        max_date_allowed=df['timestamp'].max(),
        date=df['timestamp'].max() 
    ),

    dcc.Graph(id='forecast-graph'),

    dcc.Graph(id='vehicle-type-graph'),

    html.Div(id='vehicle-output', style={'font-size': '20px', 'margin-top': '20px'})
])

@app.callback(
    [Output('forecast-graph', 'figure'),
     Output('vehicle-type-graph', 'figure'),
     Output('vehicle-output', 'children')],
    [Input('junction-dropdown', 'value'),
     Input('points-slider', 'value'),
     Input('timestamp-picker', 'date')]
)
def update_forecast(selected_junction, num_points, selected_timestamp):
    model = models[selected_junction]
    future = model.make_future_dataframe(periods=num_points, freq='min')  
    forecast = model.predict(future)

    forecast['yhat'] = forecast['yhat'].clip(lower=0)
    forecast['yhat_upper'] = forecast['yhat_upper'].clip(lower=0)
    forecast['yhat_lower'] = forecast['yhat_lower'].clip(lower=0)

    fig_forecast = go.Figure()
    fig_forecast.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predicted Vehicles'))
    fig_forecast.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', fill='tonexty', name='Upper Bound', line=dict(dash='dash')))
    fig_forecast.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], mode='lines', fill='tonexty', name='Lower Bound', line=dict(dash='dash')))
    fig_forecast.update_layout(title=f"Traffic Forecast for {selected_junction}", xaxis_title="Timestamp", yaxis_title="Predicted Vehicles")

    selected_time = pd.to_datetime(selected_timestamp)
    forecast_at_time = forecast[forecast['ds'] == selected_time]

    if not forecast_at_time.empty:
        predicted_vehicles = forecast_at_time['yhat'].values[0]
        output_text = f"Predicted number of vehicles at {selected_time}: {np.round(predicted_vehicles)}"
    else:
        output_text = f"No prediction available for the selected timestamp."
    df_junction = df[df['cctv_group'] == selected_junction]
    fig_vehicle_types = go.Figure()
    fig_vehicle_types.add_trace(go.Scatter(x=df_junction['timestamp'], y=df_junction['bicycle'], mode='lines+markers', name='Bicycles'))
    fig_vehicle_types.add_trace(go.Scatter(x=df_junction['timestamp'], y=df_junction['car'], mode='lines+markers', name='Cars'))
    fig_vehicle_types.add_trace(go.Scatter(x=df_junction['timestamp'], y=df_junction['bus'], mode='lines+markers', name='Buses'))
    fig_vehicle_types.add_trace(go.Scatter(x=df_junction['timestamp'], y=df_junction['truck'], mode='lines+markers', name='Trucks'))
    fig_vehicle_types.update_layout(title=f"Vehicle Types at {selected_junction}", xaxis_title="Timestamp", yaxis_title="Number of Vehicles")

    return fig_forecast, fig_vehicle_types, output_text

if __name__ == '__main__':
    app.run(debug=True)